## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Khatanga,RU,2021-09-12 02:04:59,71.9667,102.5000,46.53,56,4,3.13,clear sky
1,1,Souillac,MU,2021-09-12 02:07:37,-20.5167,57.5167,71.37,77,42,20.07,scattered clouds
2,2,Minas,UY,2021-09-12 02:07:37,-34.3759,-55.2377,55.87,93,49,11.59,scattered clouds
3,3,Shu,KZ,2021-09-12 02:07:37,43.5983,73.7614,62.22,25,8,2.37,clear sky
4,4,Mahibadhoo,MV,2021-09-12 02:07:38,3.7833,72.9667,82.18,73,38,10.27,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
clean_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Souillac,MU,2021-09-12 02:07:37,-20.5167,57.5167,71.37,77,42,20.07,scattered clouds
4,4,Mahibadhoo,MV,2021-09-12 02:07:38,3.7833,72.9667,82.18,73,38,10.27,scattered clouds
8,8,Rikitea,PF,2021-09-12 02:07:40,-23.1203,-134.9692,74.30,72,88,9.33,overcast clouds
9,9,Mahebourg,MU,2021-09-12 02:03:53,-20.4081,57.7000,71.26,76,23,21.12,few clouds
10,10,San Quintin,MX,2021-09-12 02:05:19,30.4833,-115.9500,74.10,79,0,10.60,clear sky
15,15,Nouakchott,MR,2021-09-12 02:03:05,18.0858,-15.9785,80.53,84,53,10.45,broken clouds
17,17,Bowen,AU,2021-09-12 02:07:43,-20.0167,148.2333,74.19,64,89,10.63,overcast clouds
29,29,Butaritari,KI,2021-09-12 02:07:48,3.0707,172.7902,83.61,75,60,23.22,light rain
34,34,Vila Velha,BR,2021-09-12 02:07:51,-20.3297,-40.2925,73.35,94,75,2.30,mist
35,35,Chimore,BO,2021-09-12 02:07:51,-16.9833,-65.1333,78.31,62,100,1.30,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
clean_df.count()

City_ID                273
City                   273
Country                273
Date                   273
Lat                    273
Lng                    273
Max Temp               273
Humidity               273
Cloudiness             273
Wind Speed             273
Current Description    273
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=clean_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Souillac,MU,71.37,scattered clouds,-20.5167,57.5167,
4,Mahibadhoo,MV,82.18,scattered clouds,3.7833,72.9667,
8,Rikitea,PF,74.30,overcast clouds,-23.1203,-134.9692,
9,Mahebourg,MU,71.26,few clouds,-20.4081,57.7000,
10,San Quintin,MX,74.10,clear sky,30.4833,-115.9500,
15,Nouakchott,MR,80.53,broken clouds,18.0858,-15.9785,
17,Bowen,AU,74.19,overcast clouds,-20.0167,148.2333,
29,Butaritari,KI,83.61,light rain,3.0707,172.7902,
34,Vila Velha,BR,73.35,mist,-20.3297,-40.2925,
35,Chimore,BO,78.31,overcast clouds,-16.9833,-65.1333,


In [18]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [29]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [30]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [35]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [36]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))